In [ ]:
#from ast import MatchSingleton
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots
import sys
sys.path.append('./RECOTOOLS/')
# import project_metadata
from core import system_run, survey_metadata, run_collection, feature_collection
import datalibrary
import processor
import systems.pelisystem
import pandas as pd
from pandas_tools import *
import matplotlib.pyplot as plt
from scipy import interpolate 
import scipy
from core import system_run, survey_metadata, run_collection
from mugrass.units import *
from mugrass.voxelframe_meshgrid import *

from mugrass.units import *
from mugrass.voxelframe_meshgrid import *
from mugrass.pixel_collection import *
from mugrass.linear_solver import *

import plotly.graph_objects as go
from plotting.plotly_tools import *

# dataset = [
#  {
#      "min_x": 00,
#      "max_x": 150000,
#      "label": "ROI 0",
#      "rroi": 260000
#  },
#  {
#      "min_x": 150000,
#      "max_x": 300000,
#      "label": "ROI 1",
#      "rroi": 260000
#  },
#  {
#      "min_x": 300000,
#      "max_x": 470000,
#      "label": "ROI 2",
#      "rroi": 260000
#  },
#  {
#      "min_x": -10000,
#      "max_x": 480000,
#      "label": "Full tunnel",
#      "rroi": 260000
#  }
# ]

dataset = [
 {
     "min_x": 0000,
     "max_x": 140000,
     "label": "ROI 0",
     "rroi": 0000
 }
]


for model in dataset:

# roi = model["roi"]
# label_roi = str(roi)
    rroi = model["rroi"]
# ftype = model["ftype"]
    fftype = "measured features"
    ROI_LABEL = model["label"]


    #voxel_col  = voxelframe_meshgrid(pickle_path="voxel_dataframe_craigydon_masked_xoff-28.vxl")


    pixel_col  = pixel_collection(pickle_path="pixel_dataframe.pxl")

    pixeldf = pixel_col.df
    #griddf = pd.read_csv(f"INTERMEDIATE/switched_fovlength_result_{ftype}_{roi}.csv",index_col=False)
    griddf = pd.read_csv("sartoutput.csv",index_col=False)

    pixeldf = pixeldf.loc[(pixeldf.x > model["min_x"]) & (pixeldf.x < model["max_x"])]
    griddf = griddf.loc[(griddf.x >= model["min_x"]) & (griddf.x <= model["max_x"])]
    #print(griddf)

    #roi = int(roi)
    minx = model["min_x"]
    maxx = model["max_x"]
    miny = -30*m
    maxy = 30*m
    minz = 0*m
    maxz = 40*m

    resx = 1*m
    resy = 1*m
    resz = 1*m

    nx   =  int(int((maxx-minx)/resx))+1
    ny   =  int(int((maxy-miny)/resy))+1
    nz   =  int(int((maxz-minz)/resz))+1

    binsx = np.linspace(minx,maxx,nx)
    binsy = np.linspace(miny,maxy,ny)
    binsz = np.linspace(minz,maxz,nz)



    #print(griddf)
    ngrid = griddf[ np.abs(griddf.y) < 1*m ].copy()


    import plotly.graph_objects as go
    # ngrid["prob"] = 1.0 - ((ngrid.value - 1.8*(g/cm3)) / (0.5*g/cm3))
    # ngrid.loc[ ngrid.prob < 0, "prob" ] = 0.0
    # ngrid.loc[ ngrid.prob > 1, "prob" ] = 1.0
    # print(ngrid["prob"])


    fig = make_subplots(rows=3, cols=1, 
                        shared_xaxes=True, 
                        vertical_spacing=0.0, 
                        row_heights=[100,20,20])

    datarow     = 1
    priorroirow = 2
    posrow      = 3
    shaftrow=2

    tunnel_min = minx/m
    tunnel_max = maxx/m
    tunnel_height = 6
    xmin = minx/m
    xmax = maxx/m
    xlow = xmin
    xwidth=10

    fig.layout.plot_bgcolor = 'white'
    fig.layout.paper_bgcolor = 'white'

    TUNNEL_NAME = "Belmont"

    # AddGeopticLogo(fig, "07/06/23")
    fig.update_layout(title=f"<b>{TUNNEL_NAME} Tunnel</b><br><i>Muon Projection: {ROI_LABEL}, {fftype} </i>", 
                        margin=dict(l=150,r=150,t=100,b=100))

    sgrid = ngrid[(np.abs(ngrid.y) < 3*m) & (ngrid.value > 0)].groupby("x").max().reset_index()
    gridx = (sgrid.x/m)
    gridy = (sgrid.z/m)
    gridy = gridy[gridx < xmax]
    gridx = gridx[gridx < xmax]
    gridx = gridx.tolist()
    gridy = gridy.tolist()
    gridx.append(tunnel_max)
    gridy.append(0.0)
    gridx.append(xmax)
    gridy.append(0.0)
    gridx.append(xmax)
    gridy.append(0.0)
    gridx.append(xmax)
    gridy.append(50)
    gridx.append(xmin)
    gridy.append(50)

    #print(gridx, gridy)

    gridbinsx = {
        "start": (binsx/m)-0.5,
        "end": (binsx/m)+0.5,
    #     "size": len(binsx)
    }

    gridbinsy = {
        "start": (binsz/m)-0.5,
        "end": (binsz/m)+0.5,
    #     "size": len(binsz)
    }

    #print(gridbinsx,gridbinsy)

    #print("NGRID", ngrid.value)
    fig.add_trace( go.Histogram2dContour(
                        x=ngrid.x/m,
                        y=ngrid.z/m,
                        z=ngrid.value/(g/cm3),
                        nbinsx=int(nx),
                        nbinsy=int(nz/1),
                        colorbar=dict(title='Density') ,
                        showscale = True,
                        showlegend=False,
                        ncontours=10,
                        histfunc="avg"
                    ), row=datarow, col=1)

    # fig.add_trace( go.Histogram2dContour(
    #                     x=ngrid.x/m,
    #                     y=ngrid.z/m,
    #                     z=ngrid.prob,
    #                     nbinsx=int(nx),
    #                     nbinsy=int(nz/2),
    #                     contours=dict(showlines=False),
    #                     colorbar=dict(title='Void Metric') ,
    #                     ncontours=20,
    #                     zmin=0.0,zmax=1.0,
    #                     histfunc="avg"
    #                 ), row=datarow, col=1)

    # fig.add_trace(
    #     go.Scatter( x=[tunnel_min,tunnel_max,tunnel_max,tunnel_min], y=[0,0,tunnel_height,tunnel_height], line=dict(color='gray'), showlegend=False, fill='toself', fillcolor='white' ),
    #     row=datarow, col=1
    # )

    def BuildOffsetFOV():
            return go.Scatter( x=[-2-3.2*np.tan(0.57+0.0713),-2,-2+2*np.tan(+0.0713),-2,-2+5.4*np.tan(0.57+0.0713)], 
                                y=[3.2,0,2,0,5.4], opacity=0.5, line=dict(color='red',thickness=1), showlegend=False )

    def BuildCircle():

        gridx = []
        gridy = []
        w  = 5.2
        h = 6.2

        for x in np.linspace(-w,w,100):
            z = h*np.sqrt(1 - (x/w)**2)
            gridx.append(x)
            gridy.append(z)

        return    go.Scatter( x=gridx, y=gridy, fill='toself', line=dict(color='gray'), showlegend=False, fillcolor='white' )




    def BuildFOV(pos):
        return go.Scatter( x=[pos,pos+tunnel_height*np.tan(0.65),pos-tunnel_height*np.tan(0.65),pos,pos], y=[0,tunnel_height,tunnel_height,0,2], opacity=0.5, 
                            line=dict(color='red'), 
                            marker=dict(size=1),
                            showlegend=False )

    for pos in pixeldf.x:
        fig.add_trace(BuildFOV(pos/m), row=datarow, col=1)

    fig.add_trace(
        go.Scatter( x=gridx, y=gridy, fill='toself', line=dict(color='gray'), showlegend=False, fillcolor='white' ),
        row=datarow,col=1
    )




    # fig.add_shape(type="rect",
    #     x0=xmin, y0=0,
    #     x1=xmax, y1=27,
    #     line=dict(
    #         color="Gray",
    #         width=4,
    #     ),
    #     row=datarow, col=1,
    #     layer='above'
    # )

# # ---------------------------------------------
# # Pos layout
# # ---------------------------------------------
    fig.add_shape(type='rect',
                    x0=xmin,
                    y0=0,
                    x1=xmax,
                    y1=40,
                    line=dict(color='Black',width=3),row=posrow,col=1)

    xlow = xmin
    xwidth=10
    while xlow < xmax:

        xhigh = xlow + xwidth
        fig.add_shape(type='rect',
                    x0=xlow,
                    y0=0,
                    x1=np.min([xlow+10,xmax]),
                    y1=40,
                    line=dict(color='Black',width=1),
                    fillcolor="White", 
                    row=posrow,col=1)

        fig.add_annotation(text="+" + str(xlow),textangle=270,x=xlow,y=15, row=posrow, col=1, 
                                showarrow=False, xanchor='center',yanchor='middle', bgcolor='White',
                                font=dict(size=13)
                                )
        xlow += xwidth

    fig.add_annotation(text="+" + str(xlow),textangle=270,x=xmax,y=15, row=posrow, col=1, 
                                showarrow=False, xanchor='left',yanchor='middle',
                                font=dict(size=13))


    fig.add_annotation(text="Distance [m]   ",textangle=0,x=xmin,y=15, row=posrow,
                                showarrow=False,  col=1, xanchor='right',yanchor='middle',
                                font=dict(size=15), xref='paper', yref='y')

    fig.update_xaxes(row=posrow,col=1,showgrid=False,visible=False,zeroline=False)
    fig.update_yaxes(row=posrow,col=1,showgrid=False,visible=False,zeroline=False,range=[0,30])

    #fig.show()
    # ------------------------------
    # SHAFT ROI
    # ------------------------------

    fig.add_annotation(text="ROI   ",textangle=0,x=xmin,y=15, row=shaftrow,
                                showarrow=False,  col=1, xanchor='right',yanchor='middle',
                                font=dict(size=15), xref='paper', yref='y')

    portal = "84mi 14ch"
    shafts_chain = ["302mi 26.78ch"]

    def abschainage(ins):
        vals = ins.replace("mi","").replace("ch","").split(" ")
        mileage = float(vals[0])
        chains = float(vals[1])
        print("mileage", mileage,chains)
        return mileage*80.0+chains

    def convertpos(chainage,portal):
        portc = abschainage(portal)
        absc = abschainage(chainage)
        difc = absc-portc
        return difc*20

    shafts = [int(rroi)/1000]

    # for chainage in shafts_chain:
    #     shafts.append(convertpos(chainage,portal))

    shaft_widths=[4]
    shaft_labels = [ROI_LABEL]

    rois = [[-2+int(rroi)/1000,+2+int(rroi)/1000]]

    # for i, chains in enumerate(rois_chain):


    #     rois.append(roi)

    for i, roi in enumerate(rois):
        fig.add_shape(type='rect',
                    x0=roi[0],
                    y0=0,
                    x1=roi[1],
                    y1=30,
                    fillcolor='Gray',
                    line=dict(color='Black',width=1),row=shaftrow,col=1)

        fig.add_annotation(text=shaft_labels[i],textangle=270,x=roi[1],y=15, row=shaftrow, col=1, 
                                showarrow=False, xanchor='left',yanchor='middle',
                                font=dict(size=13)
                                )

    for i, shaft in enumerate(shafts):
        wid = shaft_widths[i]
        lab = shaft_labels[i]

        fig.add_shape(type='line',
                    x0=shaft,
                    y0=0,
                    x1=shaft,
                    y1=30,
                    line=dict(color='Black',dash='dash',width=1),row=datarow,col=1)

        fig.add_shape(type='line',
                x0=shaft,
                y0=-30,
                x1=shaft,
                y1=30,
                line=dict(color='Black',dash='dash',width=1),row=posrow,col=1)

        fig.add_shape(type='line',
                    x0=shaft,
                    y0=0,
                    x1=shaft,
                    y1=30,
                    line=dict(color='Black',width=1),row=shaftrow,col=1)

        fig.add_shape(type='line',
                    x0=shaft-wid/2.0,
                    y0=0,
                    x1=shaft-wid/2.0,
                    y1=30,
                    line=dict(color='Black',dash='dot',width=1),row=shaftrow,col=1)

        fig.add_shape(type='line',
                    x0=shaft+wid/2.0,
                    y0=0,
                    x1=shaft+wid/2.0,
                    y1=30,
                    line=dict(color='Black',dash='dot',width=1),row=shaftrow,col=1)

    fig.add_shape(type='rect',
                    x0=xmin,
                    y0=0,
                    x1=xmax,
                    y1=30,
                    line=dict(color='Black',width=2),row=shaftrow,col=1)
    fig.update_yaxes(row=shaftrow,col=1,showgrid=False,range=[0,30],visible=False,zeroline=False,linewidth=4)






    fig.update_yaxes(range=[0.0,40], title="Elevation from Track [m]", ticks='outside', mirror=True, row=datarow,col=1)
    fig.update_xaxes(range=[xmin,xmax], title="Distance [m]", ticks='outside', mirror=True, row=posrow,col=1)
    fig.write_image(f"FIGURES/final_characterization_{ROI_LABEL}_projections.pdf",width=2048, height=1024, scale=2)
    #fig.write_html(f"FIGURES/final_characterization_{ftype}_{label_roi}_projections.html")


    fig.show()

    geoptic_plot_save(fig)


